The goal will be to construct an ETL pipeline that loads the data, performs some transformations, and writes the data to a database (and Google Cloud!).

Create a new pipeline, call it green_taxi_etl
Add a data loader block and use Pandas to read data for the final quarter of 2020 (months 10, 11, 12).
You can use the same datatypes and date parsing methods shown in the course.
BONUS: load the final three months using a for loop and pd.concat
Add a transformer block and perform the following:
Remove rows where the passenger count is equal to 0 or the trip distance is equal to zero.
Create a new column lpep_pickup_date by converting lpep_pickup_datetime to a date.
Rename columns in Camel Case to Snake Case, e.g. VendorID to vendor_id.
Add three assertions:
vendor_id is one of the existing values in the column (currently)
passenger_count is greater than 0
trip_distance is greater than 0
Using a Postgres data exporter (SQL or Python), write the dataset to a table called green_taxi in a schema mage. Replace the table if it already exists.
Write your data as Parquet files to a bucket in GCP, partioned by lpep_pickup_date. Use the pyarrow library!
Schedule your pipeline to run daily at 5AM UTC.

In [2]:
import pandas as pd
import requests


In [31]:
url=['https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2020-10.csv.gz',
         'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2020-11.csv.gz',
         'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2020-12.csv.gz']
df=pd.concat([pd.read_csv(f,compression='gzip') for f in url])


/tmp/ipykernel_10141/2979693338.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.concat([pd.read_csv(f,compression='gzip') for f in url])
/tmp/ipykernel_10141/2979693338.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.concat([pd.read_csv(f,compression='gzip') for f in url])
/tmp/ipykernel_10141/2979693338.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.concat([pd.read_csv(f,compression='gzip') for f in url])


In [32]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-10-01 00:31:19,2020-10-01 00:34:55,N,1.0,7,7,1.0,0.79,5.0,0.5,0.5,1.58,0.0,NaN,0.3,7.88,1.0,1.0,0.0
1,2.0,2020-10-01 00:42:12,2020-10-01 00:43:51,N,1.0,179,7,1.0,0.50,4.0,0.5,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0
2,2.0,2020-10-01 00:53:09,2020-10-01 00:55:39,N,1.0,179,223,1.0,0.60,4.0,0.5,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0
3,1.0,2020-10-01 00:12:29,2020-10-01 00:20:08,N,1.0,134,216,2.0,4.40,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,2.0,1.0,0.0
4,1.0,2020-10-01 00:32:38,2020-10-01 00:43:02,N,1.0,82,7,1.0,2.90,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2.0,1.0,0.0


In [33]:
#delete rows where passenger count is 0 
df=df[df['passenger_count']>0]
df=df[df['trip_distance']>0]

In [27]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-10-01 00:31:19,2020-10-01 00:34:55,N,1.0,7,7,1.0,0.79,5.0,0.5,0.5,1.58,0.0,NaN,0.3,7.88,1.0,1.0,0.0
1,2.0,2020-10-01 00:42:12,2020-10-01 00:43:51,N,1.0,179,7,1.0,0.50,4.0,0.5,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0
2,2.0,2020-10-01 00:53:09,2020-10-01 00:55:39,N,1.0,179,223,1.0,0.60,4.0,0.5,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0
3,1.0,2020-10-01 00:12:29,2020-10-01 00:20:08,N,1.0,134,216,2.0,4.40,13.5,0.5,0.5,0.00,0.0,NaN,0.3,14.80,2.0,1.0,0.0
4,1.0,2020-10-01 00:32:38,2020-10-01 00:43:02,N,1.0,82,7,1.0,2.90,10.5,0.5,0.5,0.00,0.0,NaN,0.3,11.80,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46287,2.0,2020-12-31 23:08:35,2020-12-31 23:13:00,N,1.0,41,74,1.0,0.79,5.0,0.5,0.5,0.00,0.0,NaN,0.3,6.30,2.0,1.0,0.0
46288,2.0,2020-12-31 23:11:04,2020-12-31 23:20:08,N,5.0,41,116,2.0,2.31,12.0,0.0,0.0,0.00,0.0,NaN,0.3,12.30,2.0,2.0,0.0
46289,2.0,2020-12-31 23:24:33,2020-12-31 23:40:24,N,5.0,116,119,1.0,5.73,20.0,0.0,0.0,0.00,0.0,NaN,0.3,20.30,2.0,2.0,0.0
46290,2.0,2020-12-31 23:13:59,2020-12-31 23:16:20,N,1.0,42,42,1.0,0.31,3.5,0.5,0.5,0.00,0.0,NaN,0.3,4.80,2.0,1.0,0.0


In [34]:
#Create a new column lpep_pickup_date by converting lpep_pickup_datetime to a date.
df['lpep_pickup_date']=pd.to_datetime(df['lpep_pickup_datetime']).dt.date


In [38]:
#Rename columns in Camel Case to Snake Case, e.g. VendorID to vendor_id.
print(df.columns)


Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'lpep_pickup_date'],
      dtype='object')


In [40]:
#rename columns to camel case
df.columns=['vendor_id', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'ratecode_id', 'pulocation_id', 'dolocation_id',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'lpep_pickup_date']
print(df.columns)

Index(['vendor_id', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'ratecode_id', 'pulocation_id', 'dolocation_id',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'lpep_pickup_date'],
      dtype='object')


In [41]:
#What are the existing values of VendorID in the dataset?
df['vendor_id'].unique()


array([2., 1.])

In [42]:
#add asertion : vendor_id is one of the existing values in the column  1 or 2
assert df['vendor_id'].isin([1,2]).all()
#asserrtion passenger_count is greater than 0
assert (df['passenger_count']>0).all()
#assertion trip_distance is greater than 0
assert (df['trip_distance']>0).all()

In [ ]:
#using py arrow transform df to parquet and partition by lpep_pickup_date
import pyarrow as pa
import pyarrow.parquet as pq
import os

#transform df to parquet
table = pa.Table.from_pandas(df)
pq.write_table(table, 'data.parquet')